Multi-fidelity Modeling and Experimental Design (Active Learning)

In [ ]:
# General imports

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
%matplotlib inline
import GPy
import emukit.multi_fidelity
import emukit.test_functions
from emukit.model_wrappers.gpy_model_wrappers import GPyMultiOutputWrapper
from emukit.multi_fidelity.models import GPyLinearMultiFidelityModel
from emukit.multi_fidelity.convert_lists_to_array import convert_x_list_to_array, convert_xy_lists_to_arrays
np.random.seed(20)

from emukit.multi_fidelity.models.non_linear_multi_fidelity_model import make_non_linear_kernels, NonLinearMultiFidelityModel

In [ ]:
data = np.genfromtxt('Ge77Rates_v1.csv',
                     dtype=None,
                     delimiter=',',
                    skip_header = 1)
print(data)

In [ ]:
x_train_l, x_train_h, y_train_l, y_train_h = ([],[],[],[])
n_low_fidelity_points = 0
n_high_fidelity_points = 0
error_low = []
error_hi = []
for entry in data:
    if entry[2] == b'LF':
        x_train_l.append(entry[1])
        y_train_l.append(entry[-4])
        error_low.append(entry[-3])
    else:
        x_train_h.append(entry[1])
        y_train_h.append(entry[-4])
        error_hi.append(entry[-3])

x_train_l, x_train_h, y_train_l, y_train_h = (np.atleast_2d(x_train_l).T, np.atleast_2d(x_train_h).T, np.atleast_2d(y_train_l).T, np.atleast_2d(y_train_h).T)

X_train, Y_train = convert_xy_lists_to_arrays([x_train_l, x_train_h], [y_train_l, y_train_h])


plt.figure(figsize=(12, 8))
plt.scatter(x_train_l, y_train_l, color='b', s=40)
plt.scatter(x_train_h, y_train_h, color='r', s=40)
plt.xlabel('x')
plt.ylabel('f (x)')
plt.xlim([0, 260])
plt.legend(['Low fidelity', 'High fidelity'])
plt.title('High and low fidelity functions');

In [ ]:
# Construct a linear multi-fidelity model

kernels = [GPy.kern.RBF(1), GPy.kern.RBF(1)]
lin_mf_kernel = emukit.multi_fidelity.kernels.LinearMultiFidelityKernel(kernels)
gpy_lin_mf_model = GPyLinearMultiFidelityModel(X_train, Y_train, lin_mf_kernel, n_fidelities=2)
gpy_lin_mf_model.mixed_noise.Gaussian_noise.fix(1e-6)
'''
The Low Fidelity noise level need to be independently estimated. Here I provide a guess of 5e-7
'''
gpy_lin_mf_model.mixed_noise.Gaussian_noise_1.fix(0)

## Wrap the model using the given 'GPyMultiOutputWrapper'

lin_mf_model = model = GPyMultiOutputWrapper(gpy_lin_mf_model, 2, n_optimization_restarts=20)

## Fit the model
  
lin_mf_model.optimize()
nonlin_mf_model = lin_mf_model

In [ ]:
SPLIT = 200
## Compute mean and variance predictions
x_plot = np.linspace(0, 264, SPLIT)[:, None]
X_plot = convert_x_list_to_array([x_plot, x_plot])

hf_mean_nonlin_mf_model, hf_var_nonlin_mf_model = nonlin_mf_model.predict(X_plot[:SPLIT])
hf_std_nonlin_mf_model = np.sqrt(hf_var_nonlin_mf_model)

lf_mean_nonlin_mf_model, lf_var_nonlin_mf_model = nonlin_mf_model.predict(X_plot[SPLIT:])
lf_std_nonlin_mf_model = np.sqrt(lf_var_nonlin_mf_model)


## Plot posterior mean and variance of nonlinear multi-fidelity model

plt.figure(figsize=(12,8))
plt.fill_between(x_plot.flatten(), (lf_mean_nonlin_mf_model - 1.96*lf_std_nonlin_mf_model).flatten(), 
                 (lf_mean_nonlin_mf_model + 1.96*lf_std_nonlin_mf_model).flatten(), color='g', alpha=0.3)
plt.fill_between(x_plot.flatten(), (hf_mean_nonlin_mf_model - 1.96*hf_std_nonlin_mf_model).flatten(), 
                 (hf_mean_nonlin_mf_model + 1.96*hf_std_nonlin_mf_model).flatten(), color='y', alpha=0.3)
plt.plot(x_train_l, y_train_l, 'b',marker=".")
plt.plot(x_train_h, y_train_h, 'r',marker=".")
plt.plot(x_plot, lf_mean_nonlin_mf_model, '--', color='g')
plt.plot(x_plot, hf_mean_nonlin_mf_model, '--', color='y')
plt.scatter(x_train_h, y_train_h, color='r')
plt.xlabel('Radii')
plt.ylabel('Ge77 Production Rate')
plt.xlim(0, 250)
plt.legend(['Low Fidelity', 'High Fidelity', 'Predicted Low Fidelity', 'Predicted High Fidelity'])
plt.title('linear multi-fidelity model fit to low and high fidelity functions');

# Acqusition Curve
- The acquisition curve is an important part of the active learning process. The next step we try using HF simulation dependes on where the acquisiton function takes its maximal value.
- Define a parameter space (here we only have a single parameter radius), we need to add another parameter fidelity into our data, this parameter is always 1, meaning that we always run acquisition function on the high fidelity (1) space.
- Note: it is important to deine the lower and upper range of our optimization parameter. Looking at the previous plot, anything below 80cm is probably unphysical, therefore we should not waste any attempt there. I selected a region of 90-250 to run the acquisition function. Selecting the wrong range could significantly change the shape of acquisition function.

In [ ]:
from emukit.experimental_design.acquisitions import ModelVariance,IntegratedVarianceReduction
from emukit.core import ParameterSpace, ContinuousParameter,DiscreteParameter
SPLIT = 200
low = 90
high =250
## Compute mean and variance predictions
x_plot = np.linspace(low, high, SPLIT)[:, None]
X_plot = convert_x_list_to_array([x_plot, x_plot])
parameter_space = ParameterSpace([ContinuousParameter('x1', low, high), DiscreteParameter("f",[1])])
us_acquisition = IntegratedVarianceReduction(nonlin_mf_model, parameter_space)
acq = us_acquisition.evaluate(X_plot[SPLIT:])
plt.plot(np.linspace(low, high, SPLIT),acq/acq.max())

Here we run a gradient-based optimizer over the acquisition function to find the next point to attempt

In [ ]:
from emukit.core.optimization import GradientAcquisitionOptimizer
optimizer = GradientAcquisitionOptimizer(parameter_space)
x_new, _ = optimizer.optimize(us_acquisition)

In [ ]:
print(x_new)

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(x_plot, acq / np.max(acq), "green", label="US")
plt.axvline(x_new[0,0], color="red", label="x_next", linestyle="--")
plt.legend(loc=1)
plt.xlabel(r"$x$")
plt.ylabel(r"$f(x)$")
plt.grid(True)
plt.show()

Let's say we attempted another HF simulation at `x = 240.01400146`, and obtain the Ge77 production rate to be `0.012`. Then we can prepare this new HF dataset and add it back to the model.

In [ ]:
x_new_data = np.array([[240.01400146,1.0]])
y_new_data = np.array([[0.012]])
X_train = np.append(X_train,x_new_data,axis=0)
Y_train = np.append(Y_train,y_new_data,axis=0)
nonlin_mf_model.set_data(X_train, Y_train)

SPLIT = 200
## Compute mean and variance predictions
x_plot = np.linspace(0, 264, SPLIT)[:, None]
X_plot = convert_x_list_to_array([x_plot, x_plot])

hf_mean_nonlin_mf_model, hf_var_nonlin_mf_model = nonlin_mf_model.predict(X_plot[:SPLIT])
hf_std_nonlin_mf_model = np.sqrt(hf_var_nonlin_mf_model)

lf_mean_nonlin_mf_model, lf_var_nonlin_mf_model = nonlin_mf_model.predict(X_plot[SPLIT:])
lf_std_nonlin_mf_model = np.sqrt(lf_var_nonlin_mf_model)


## Plot posterior mean and variance of nonlinear multi-fidelity model

plt.figure(figsize=(12,8))
plt.fill_between(x_plot.flatten(), (lf_mean_nonlin_mf_model - 1.96*lf_std_nonlin_mf_model).flatten(), 
                 (lf_mean_nonlin_mf_model + 1.96*lf_std_nonlin_mf_model).flatten(), color='g', alpha=0.3)
plt.fill_between(x_plot.flatten(), (hf_mean_nonlin_mf_model - 1.96*hf_std_nonlin_mf_model).flatten(), 
                 (hf_mean_nonlin_mf_model + 1.96*hf_std_nonlin_mf_model).flatten(), color='y', alpha=0.3)
plt.plot(x_train_l, y_train_l, 'b',marker=".")
plt.plot(x_train_h, y_train_h, 'r',marker=".")
plt.scatter([x_new_data[0,0]], [y_new_data[0,0]],color="orange")
plt.plot(x_plot, lf_mean_nonlin_mf_model, '--', color='g')
plt.plot(x_plot, hf_mean_nonlin_mf_model, '--', color='y')
plt.scatter(x_train_h, y_train_h, color='r')
plt.xlabel('Radii')
plt.ylabel('Ge77 Production Rate')
plt.xlim(0, 250)
plt.legend(['Low Fidelity', 'High Fidelity','Predicted Low Fidelity', 'Predicted High Fidelity',"Newly Attempted HF Point"])
plt.title('linear multi-fidelity model fit to low and high fidelity functions');

Now we rerun the acquisition function and gradient finding function, we will see that it suggests a next point to try

In [ ]:
us_acquisition = IntegratedVarianceReduction(nonlin_mf_model, parameter_space)
acq = us_acquisition.evaluate(X_plot[SPLIT:])
optimizer = GradientAcquisitionOptimizer(parameter_space)
x_new, _ = optimizer.optimize(us_acquisition)
print(x_new)
plt.figure(figsize=(12, 8))
plt.plot(x_plot, acq / np.max(acq), "green", label="IVR")
plt.axvline(x_new[0,0], color="red", label="x_next", linestyle="--")
plt.legend(loc=1)
plt.xlabel(r"$x$")
plt.ylabel(r"$f(x)$")
plt.grid(True)
plt.show()

Now it suggests us to attempt 161cm as the next step. We can keep looping through these procedure until we are confident with our final result.